# 简易的QA rag 实现

In [1]:
import shutup
shutup.please()


from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model='qwen2:7b', temperature=0)

In [2]:
# 构建简单文本库
from langchain.schema import Document

# corpus 语料库
corpus = [
    '武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。',  
    '武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。',
    '武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。',
    '湖北省，简称“鄂”，别名楚、荆楚，中华人民共和国省级行政区，省会武汉。',
    '湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。',
    '截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。',
]

documents = [Document(page_content=cp) for cp in corpus]
documents

[Document(page_content='武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。'),
 Document(page_content='武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。'),
 Document(page_content='武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。'),
 Document(page_content='湖北省，简称“鄂”，别名楚、荆楚，中华人民共和国省级行政区，省会武汉。'),
 Document(page_content='湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。'),
 Document(page_content='截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。')]

有了文档之后，我们可以使用QAchain来实现一个简易的rag

In [3]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm=llm, chain_type='stuff', verbose=True)

question = '武汉三镇是哪三个？'
result = chain.run(input_documents=documents, question=question)
result



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。

武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。

武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。

湖北省，简称“鄂”，别名楚、荆楚，中华人民共和国省级行政区，省会武汉。

湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。

截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。
Human: 武汉三镇是哪三个？

> Finished chain.

> Finished chain.


'武汉三镇指的是武昌、汉口和汉阳。这三个城市在长江中游交汇处隔江鼎立，共同组成了今天的武汉市。'

In [4]:
question = '湖北省常住人口是多少？'
result = chain.run(input_documents=documents, question=question)
result



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。

武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。

武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。

湖北省，简称“鄂”，别名楚、荆楚，中华人民共和国省级行政区，省会武汉。

湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。

截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。
Human: 湖北省常住人口是多少？

> Finished chain.

> Finished chain.


'湖北省常住人口是5844万人。'

In [5]:
# 再试下不相关的问题
question = '河南省常住人口是多少？'
result = chain.run(input_documents=documents, question=question)
result



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。

武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。

武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。

湖北省，简称“鄂”，别名楚、荆楚，中华人民共和国省级行政区，省会武汉。

湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。

截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。
Human: 河南省常住人口是多少？

> Finished chain.

> Finished chain.


'关于河南省的常住人口信息在给定的内容中没有提供，因此我无法直接回答这个问题。通常获取这类数据需要查找最新的官方统计数据或报告。'

可以看到，根据给定的简易文档，大模型的rag回答效果是不错的。

此外，chain_type有其他参数：

- stuff
    - 将所有文档作为context放入到Prompt中，传递到LLM获取答案，适合小文档。

- refine
 -  上一轮的输出作为下一轮的输入，迭代更新其答案，以获得最好的最终结果。

- map_reduce
    - 将LLM链应用于每个单独的文档，最后合并汇总，得到答案。

- map_rerank
    - 每个文档上运行一个初始提示，再给对应输出给一个分数，返回得分最高的回答。


接下来来试试各个参数的含义：

In [6]:
chain = load_qa_chain(llm=llm, chain_type='refine', verbose=True)
question = '武汉三镇是哪三个？'
result = chain.run(input_documents=documents, question=question)
result



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Context information is below.
------------
武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。
------------
Given the context information and not prior knowledge, answer any questions
Human: 武汉三镇是哪三个？

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 武汉三镇是哪三个？
AI: 武汉三镇分别是汉口（今江岸区）、武昌（今洪山区）和汉阳。这三个区域在历史上各自发展，形成现在的武汉市。其中，汉口曾是独立的城镇并一度成为湖北省的首府；武昌则因其位于长江与东湖之间、濒临长江的部分而得名，并且拥有悠久的历史文化；汉阳则是以地理位置靠近汉江而闻名。三镇通过水路相连，共同构成了武汉这座现代化城市的基础。
Human: We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。
------------
Given the new context, refine the original answer to better answer the question. If the context isn't useful, return the original answer.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formattin

'原始答案：武汉三镇指的是中国湖北省的三个主要区域：汉口（今江岸区）、武昌（今洪山区）和汉阳。这些城市位于长江中游，具体而言就是在湖北省境内的长江流域，它们隔江鼎立的位置使得这三个城市在历史上具有独特的战略地位与文化特性。武汉作为湖北省的省会，在这三镇之中起到了核心与联系的作用。\n\n基于新上下文的信息更新后的答案：武汉三镇指的是中国湖北省内三个主要区域，即汉口（今江岸区）、武昌（今洪山区）和汉阳。这三个城市位于长江中游地区，并在湖北省境内沿长江流域分布。历史上，由于它们隔江鼎立的位置，这三座城市均拥有显著的地理位置优势和独特的文化特点。作为湖北省的省会，武汉在这三个区域之间扮演着重要角色。\n\n更新后的答案通过引入新提供的上下文（如湖北省的人口、地区生产总值等信息）对原始回答进行了丰富，并保持了关于武汉三镇历史地位与地理特性的核心描述不变。'

In [12]:
chain = load_qa_chain(llm=llm, chain_type='map_reduce', verbose=True)

question = '湖北省的常驻人口有多少？'
result = chain.run(input_documents=documents, question=question)
result



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。
Human: 湖北省的常驻人口有多少？
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。
Human: 湖北省的常驻人口有多少？
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。
Human: 湖北省的常驻人口有多少？
Prompt after formatting:
System: Use the following portion of a long do


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Given the following extracted parts of a long document and a question, create a final answer. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
______________________
文本中没有提供关于湖北省常驻人口的具体数字信息。

武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人。

因此，湖北省的常驻人口与上述信息直接相关。然而，仅凭提供的数据无法精确计算整个湖北省的常驻人口数量，因为上述信息只提供了武汉市的具体数据。要获取湖北省全境的常驻人口数据，需要额外的数据来源或详细统计报告。

文档中的文本与回答提供的问题无关。因此，没有相关的文本可以提供以解答关于“湖北省的常驻人口”的问题。

请注意，为了提供准确的答案，我需要一段包含有湖北省常住人口数据的相关文本或信息。然而，在当前给出的信息中并没有这样的内容。

文档中并未提供湖北省常驻人口数量的相关信息。

文档中的文本与回答提供的问题无关，因此没有相关文本可供返回。

截至2022年末，湖北省常住人口5844万人。
Human: 湖北省的常驻人口有多少？

> Finished chain.

> Finished chain.


'湖北省的常驻人口为5844万人。'

In [14]:
chain = load_qa_chain(llm=llm, chain_type='map_rerank', verbose=True)

question = '湖北省常驻人口是多少？'
result = chain.run(input_documents=documents, question=question)
result



> Entering new MapRerankDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
--

'湖北省常驻人口是5844万人。'

在 token 数量不超限的情况下，stuff 的效果最好。对于简单问答场景，map_rerank 也是不错的选择。

此外，我们还可以实现对文档的过滤，比如问武汉市的内容，那湖北省的片段实际上是可以被过滤的。过滤的方法详见[FAISS召回](./10_faiss.ipynb)。